IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

IMPORTING THE TRAINING DATASET

In [2]:
dataset = pd.read_csv('train.csv')

DATA PRE PROCESSING-DROPPING COLUMNS OF DATAS WHICH ARE NOT STRONGLY RELATED TO 1

In [3]:
dataset.drop(dataset.columns[[0,13, 14, 16,17,18,19,10,11,14,15,12]], axis = 1, inplace = True)

DATA PRE PROCESSING-SPLITTING DATASET INTO INDEPENDENT AND DEPENDENT VARIABLES


In [4]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

CREATING LABEL ENCODER

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()

DATA PRE PROCESSING-ENCODING DATA

In [6]:
#gender
X[:,0]=le.fit_transform(X[:,0])
#s11
X[:,1]=le.fit_transform(X[:,1])
#s12
X[:,2]=le.fit_transform(X[:,2])
#s16
X[:,3]=le.fit_transform(X[:,3])
#s17
X[:,4]=le.fit_transform(X[:,4])
X[:,5]=le.fit_transform(X[:,5])
#s18
X[:,6]=le.fit_transform(X[:,6])
X=pd.DataFrame(X)
X[8].replace(to_replace = 'l', value =float(1),inplace=True)
X[8].replace(to_replace = 'o', value =float(0),inplace=True)
X[8]=X[8].astype(np.float64)

SPLITTING DATASET INTO TRAINING AN TESTSET

In [7]:
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.3, random_state = 0)

SCALING THE DATASET

In [8]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)

CREATING THE NEURAL NETWORK MODEL

In [9]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'] )

In [ ]:
FITTING THE DATASET

In [10]:
ann.fit(XTrain, yTrain, batch_size=32, epochs=100)

Epoch 1/100
620/620 [==============================] - 1s 663us/step - loss: 0.4099 - accuracy: 0.8250
Epoch 2/100
620/620 [==============================] - 0s 668us/step - loss: 0.3347 - accuracy: 0.8388
Epoch 3/100
620/620 [==============================] - 0s 661us/step - loss: 0.3283 - accuracy: 0.8678
Epoch 4/100
620/620 [==============================] - 0s 666us/step - loss: 0.3249 - accuracy: 0.8702
Epoch 5/100
620/620 [==============================] - 0s 665us/step - loss: 0.3225 - accuracy: 0.8706
Epoch 6/100
620/620 [==============================] - 0s 653us/step - loss: 0.3206 - accuracy: 0.8718
Epoch 7/100
620/620 [==============================] - 0s 652us/step - loss: 0.3189 - accuracy: 0.8723
Epoch 8/100
620/620 [==============================] - 0s 656us/step - loss: 0.3175 - accuracy: 0.8720
Epoch 9/100
620/620 [==============================] - 0s 692us/step - loss: 0.3161 - accuracy: 0.8730
Epoch 10/100
620/620 [==============================] - 0s 652us/step - l

PREDICTION RESULT

In [12]:
y_pred = ann.predict(XTest)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), yTest.reshape(len(yTest), 1)), 1))

266/266 [==============================] - 0s 476us/step
[[1 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


CONFUSION MATRIX AND ACCURACY

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(yTest, y_pred)
print(cm)
accuracy_score(yTest, y_pred)*100

[[6809  279]
 [ 777  632]]


87.57208426503472

TEST DATASET LOADING

In [14]:
testData=pd.read_csv('test.csv')

TEST DATASET PREPROCESSING

In [16]:
testData.drop(testData.columns[[0,13, 14, 16,17,18,19,10,11,14,15,12]], axis = 1, inplace = True)
X = testData.iloc[:, :].values
#gender
X[:,0]=le.fit_transform(X[:,0])
#gender
X[:,1]=le.fit_transform(X[:,1])
#s11
X[:,2]=le.fit_transform(X[:,2])
#s12
X[:,3]=le.fit_transform(X[:,3])
#s16
X[:,4]=le.fit_transform(X[:,4])
#s17
X[:,5]=le.fit_transform(X[:,5])
#s18
X[:,6]=le.fit_transform(X[:,6])

X=pd.DataFrame(X)
X[8].replace(to_replace = 'l', value ='1',inplace=True)
X[8].replace(to_replace = 'o', value ='0',inplace=True)
X[8]=X[8].astype(np.float64)

SCALING TEST DATASET


In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
y_pred = ann.predict(X)

2659/2659 [==============================] - 1s 477us/step


GETTING VALUES OF PREDICTIONS FOR THE TEST DATASET

In [18]:
for cell in np.nditer(y_pred, op_flags=['readwrite']):
    if cell[...] > 0.5:
      cell[...] = 1
    else:
      cell[...] = 0

CREATING THE SUBMISSION FILE

In [ ]:
test2_dataset = pd.read_csv('test.csv')
test2_dataset['label'] = y_pred
submission = test2_dataset[["id", "label"]]
submission.to_csv("submission2.csv", index = False)
